In [1]:
from collections import deque
from itertools import chain
# ===== vsualzie ===== 
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle
from matplotlib.pyplot import MultipleLocator


In [2]:
def readGrid(path):
    with open(f"../assets/data/Kuroshuto/{path}.txt") as f:
        num = f.readline()
        m, n = num.split(" ")[0], num.split(" ")[1]
        grid = f.readlines()
        res = [g.strip().split(" ") for g in grid]
        return int(m), int(n), res

if __name__ == "__main__":
    m, n, grid = readGrid("problems/13_12x12")
    
    for g in grid:
        print(g)

['-', '-', '-', '-', '1', '-', '5', '1', '-', '-', '-', '-']
['4', '-', '-', '-', '5', '2', '-', '-', '-', '-', '-', '-']
['-', '-', '1', '-', '-', '-', '-', '2', '-', '-', '-', '-']
['-', '2', '-', '5', '-', '5', '-', '4', '4', '-', '-', '-']
['4', '-', '-', '5', '-', '-', '-', '-', '-', '-', '1', '-']
['-', '3', '-', '5', '2', '-', '2', '2', '-', '-', '-', '4']
['-', '-', '-', '-', '2', '4', '-', '-', '-', '-', '-', '8']
['-', '1', '-', '-', '4', '5', '-', '2', '1', '-', '2', '-']
['5', '-', '4', '-', '-', '-', '1', '2', '6', '5', '-', '3']
['-', '-', '-', '-', '-', '5', '-', '7', '4', '3', '-', '-']
['-', '-', '-', '1', '2', '-', '-', '-', '-', '-', '-', '-']
['1', '-', '-', '-', '-', '-', '-', '2', '2', '-', '3', '-']


In [ ]:
from ortools.sat.python import cp_model as cp
def kuroshuto_solver(grid, X, Y):
    model = cp.CpModel()
    x = {}
    for i in range(X):
        for j in range(Y):
            if grid[i][j] in ["-"]:
                x[i, j] = model.NewBoolVar(name = f"x[{i}, {j}]")
            else:
                x[i, j] = 0
    for i in range(X):
        for j in range(Y):
            cand_pos = [] 
            if grid[i][j] in "0123456789":
                dist = int(grid[i][j])
                if i - dist >= 0 and grid[i - dist][j] not in "0123456789":
                    cand_pos.append((i - dist, j))
                if j - dist >= 0 and grid[i][j - dist] not in "0123456789":
                    cand_pos.append((i, j - dist))
                if i + dist < X and grid[i + dist][j] not in "0123456789":
                    cand_pos.append((i + dist, j))
                if j + dist < Y and grid[i][j + dist] not in "0123456789":
                    cand_pos.append((i, j + dist))
                
                model.Add(sum([x[p] for p in cand_pos]) == 1)
    
    solver = cp.CpSolver()
    status = solver.Solve(model)
    if status == cp.OPTIMAL:
        for i in range(X):
            for j in range(Y):
                if grid[i][j] not in "0123456789":
                    if solver.Value(x[i, j]) > 1e-1:
                        print("X", end=" ")
                    else:
                        print("O", end=" ")
                else:
                    print(grid[i][j], end=" ")
            print()
        print()

        print("NumConflicts:", solver.NumConflicts())
        print("NumBranches:", solver.NumBranches())
        print("WallTime:", solver.WallTime())
    else:
        print("Something Wrong!")
if __name__ == "__main__":
    m, n, grid = readGrid("problems/13_12x12")
    kuroshuto_solver(grid, m, n)
    
# x - - x - - - - - - - x
# - x - - - - - x - x - -
# - - - x - x - - x - - -
# x - - - x - x - - x - x
# - - x - - - - - x - - -
# x - - - - x - - - - x -
# - - - x - - x - - - - -
# - - - - - - - - - x - -
# - x - x - - - - - - x -
# x - - - x - x - - - - x
# - - x - - - - x - - - -
# - x - - - - x - - x - x

X O O X 1 O 5 1 O O O X 
4 X X O 5 2 O X O X O O 
O O 1 O O X O 2 X O O O 
X 2 O 5 X 5 X 4 4 X O X 
4 O X 5 O O O O X O 1 O 
X 3 O 5 2 O 2 2 O X X 4 
O O X X 2 4 O X O O O 8 
O 1 O O 4 5 O 2 1 X 2 O 
5 X 4 X O O 1 2 6 5 X 3 
X O O O X 5 X 7 4 3 O X 
O O O 1 2 O X O O O O O 
1 X O X O O X 2 2 X 3 X 

NumConflicts: 0
NumBranches: 0
WallTime: 0.010576
